In [2]:
import sys
import os
print("\n".join(sys.path))


C:\Users\udayp\AppData\Roaming\uv\python\cpython-3.10.18-windows-x86_64-none\python310.zip
C:\Users\udayp\AppData\Roaming\uv\python\cpython-3.10.18-windows-x86_64-none\DLLs
C:\Users\udayp\AppData\Roaming\uv\python\cpython-3.10.18-windows-x86_64-none\lib
C:\Users\udayp\AppData\Roaming\uv\python\cpython-3.10.18-windows-x86_64-none
d:\Projects\AI_Trip_Planner\env

d:\Projects\AI_Trip_Planner\env\lib\site-packages
d:\Projects\AI_Trip_Planner\env\lib\site-packages\win32
d:\Projects\AI_Trip_Planner\env\lib\site-packages\win32\lib
d:\Projects\AI_Trip_Planner\env\lib\site-packages\Pythonwin


In [24]:
# Imports
import requests
import json

# Define the class
class WeatherForecastTool:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://api.openweathermap.org/data/2.5"

    def get_current_weather(self, place: str):
        """Get current weather of a place"""
        try:
            url = f"{self.base_url}/weather"
            params = {
                "q": place,
                "appid": self.api_key,
                "units" : "metric"  # use "imperial" for Fahrenheit
            }
            response = requests.get(url, params=params)
            return response.json() if response.status_code == 200 else {}
        except Exception as e:
            raise e

    def get_forecast_weather(self, place: str):
        """Get weather forecast of a place"""
        try:
            url = f"{self.base_url}/forecast"
            params = {
                "q": place,
                "appid": self.api_key,
                "cnt": 10,          # number of forecast entries
                "units": "metric"   # use "imperial" for Fahrenheit
            }
            response = requests.get(url, params=params)
            return response.json() if response.status_code == 200 else {}
        except Exception as e:
            raise e


In [25]:
import dotenv
import os
dotenv.load_dotenv()

# Create an instance
api_key=os.getenv("OPENWEATHER_API_KEY")
weather_tool = WeatherForecastTool(api_key=api_key)

# Test current weather
city = "Datia"
print("Current Weather:")
current = weather_tool.get_current_weather(city)
if current:
    print(f"Temperature: {current['main']['temp']} °C")
    print(f"Weather: {current['weather'][0]['description']}")
else:
    print("Failed to fetch current weather.")

# Test forecast weather
print("\nForecast Weather:")
forecast = weather_tool.get_forecast_weather(city)
if forecast:
    for entry in forecast["list"]:
        print(f"{entry['dt_txt']}: {entry['main']['temp']}°C, {entry['weather'][0]['description']}")
else:
    print("Failed to fetch forecast weather.")


Current Weather:
Temperature: 30.54 °C
Weather: moderate rain

Forecast Weather:
2025-07-06 21:00:00: 30.95°C, overcast clouds
2025-07-07 00:00:00: 30.03°C, light rain
2025-07-07 03:00:00: 32.07°C, light rain
2025-07-07 06:00:00: 34.67°C, light rain
2025-07-07 09:00:00: 35.19°C, light rain
2025-07-07 12:00:00: 35.81°C, broken clouds
2025-07-07 15:00:00: 32.58°C, overcast clouds
2025-07-07 18:00:00: 31.43°C, overcast clouds
2025-07-07 21:00:00: 30.28°C, light rain
2025-07-08 00:00:00: 29.76°C, moderate rain


In [26]:
import requests
import os
import dotenv
dotenv.load_dotenv()

api_key = os.getenv("EXCHANGE_RATE_API_KEY")

class CurrencyConverter:
    def __init__(self, api_key: str):
        self.base_url = f"https://v6.exchangerate-api.com/v6/{api_key}/latest/"
    
    def convert(self, amount:float, from_currency:str, to_currency:str):
        """Convert the amount from one currency to another"""
        url = f"{self.base_url}/{from_currency}"
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception("API call failed:", response.json())
        rates = response.json()["conversion_rates"]
        if to_currency not in rates:
            raise ValueError(f"{to_currency} not found in exchange rates.")
        return amount * rates[to_currency]

In [30]:
# Import your class  # Adjust the path if it's in utils/

# Load API key from .env
import os
import dotenv
dotenv.load_dotenv()
api_key = os.getenv("EXCHANGE_RATE_API_KEY")

# Create an instance
converter = CurrencyConverter(api_key)

# Test conversion
try:
    amount = 10
    from_currency = "USD"
    to_currency = "Russian"
    
    converted = converter.convert(amount, from_currency, to_currency)
    print(f"{amount} {from_currency} = {converted:.2f} {to_currency}")
except Exception as e:
    print("❌ Error during conversion:", e)


❌ Error during conversion: Russian not found in exchange rates.


In [ ]:
import os
import dotenv
dotenv.load_dotenv()
from langchain_tavily import TavilySearch


class TavilyPlaceSearchTool:
    def __init__(self):
        self.api_key = os.getenv("TAVILAY_API_KEY")
        

    def search_attractions(self, place: str) -> str:
        """
        Searches for popular tourist attractions in the specified place using TavilySearch.
        """
        tavily_tool = TavilySearch(topic="travel-planner", include_answer="advanced")
        result = tavily_tool.invoke({
            "query": f"As a travel planner, give me the top tourist attractions in and around {place}, with descriptions, why they are popular, and any ticket or timing info."
        })
        return result.get("answer", result)

    def search_veg_restaurants(self, place: str) -> str:
        """
        Searches for vegetarian restaurants in the specified place using TavilySearch.
        """
        tavily_tool = TavilySearch(topic="travel-planner", include_answer="advanced")
        result = tavily_tool.invoke({
            "query": f"List the best vegetarian restaurants in and around {place}. Include local dishes they are known for and user recommendations."
        })
        return result.get("answer", result)

    def search_nonveg_restaurants(self, place: str) -> str:
        """
        Searches for non-vegetarian restaurants in the specified place using TavilySearch.
        """
        tavily_tool = TavilySearch(topic="travel-planner", include_answer="advanced")
        result = tavily_tool.invoke({
            "query": f"What are the best non-vegetarian restaurants in and around {place}? Mention top dishes, cuisines, and chef specialties if any."
        })
        return result.get("answer", result)

    def search_activities(self, place: str) -> str:
        """
        Searches for popular activities in the specified place using TavilySearch.
        """
        tavily_tool = TavilySearch(topic="travel-planner", include_answer="advanced")
        result = tavily_tool.invoke({
            "query": f"What are the best cultural, adventure, and local activities that a traveler must try in {place}?"
        })
        return result.get("answer", result)

    def search_transportation(self, place: str) -> str:
        """
        Searches for transportation options in the specified place using TavilySearch.
        """
        tavily_tool = TavilySearch(topic="travel-planner", include_answer="advanced")
        result = tavily_tool.invoke({
            "query": f"What are the different modes of transportation in {place} for tourists, including local transit, cabs, and long-distance travel options?"
        })
        return result.get("answer", result)


In [4]:
# test_tavily_tool.py

# Create an instance
tool = TavilyPlaceSearchTool()

# Define a place to test
place = "Udaipur"

print("\n📍 Top Attractions in", place)
print(tool.search_attractions(place))

print("\n🥗 Best Vegetarian Restaurants in", place)
print(tool.search_veg_restaurants(place))

print("\n🍗 Best Non-Vegetarian Restaurants in", place)
print(tool.search_nonveg_restaurants(place))

print("\n🎉 Top Activities to Try in", place)
print(tool.search_activities(place))

print("\n🚕 Transportation Options in", place)
print(tool.search_transportation(place))



📍 Top Attractions in Udaipur


ValidationError: 1 validation error for TavilySearchAPIWrapper
  Value error, Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [6]:
# test_tavily_tool.py
import sys
import os
sys.path.append(os.path.abspath(".."))  # or path to the project root


import os
import dotenv
dotenv.load_dotenv()

print("TAVILY_API_KEY =", os.getenv("TAVILAY_API_KEY"))  # Debug print


tool = TavilyPlaceSearchTool()
place = "Udaipur"

print("\nTop Attractions:")
print(tool.search_attractions(place))


TAVILY_API_KEY = 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

Top Attractions:


ValidationError: 1 validation error for TavilySearchAPIWrapper
  Value error, Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [1]:
import os
import dotenv
dotenv.load_dotenv()

# Optional debug print
print("TAVILY_API_KEY =", os.getenv("TAVILAY_API_KEY"))


TAVILY_API_KEY = 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


In [ ]:
from langchain_tavily import TavilySearch
import os

class TavilyPlaceSearchTool:
    def __init__(self):
        self.api_key = os.getenv("TAVILY_API_KEY")

    def search_attractions(self, place: str) -> str:
        tavily_tool = TavilySearch(
            topic="travel-planner",
            include_answer="advanced",
            tavily_api_key=self.api_key  # ✅ explicitly pass the key here
        )
        result = tavily_tool.invoke({
            "query": f"As a travel planner, give me the top tourist attractions in and around {place}, with descriptions, why they are popular, and any ticket or timing info."
        })
        return result.get("answer", result)

    # Repeat this pattern for the rest


In [10]:
tool = TavilyPlaceSearchTool()

# Define a place to test
place = "Udaipur"

print("\n📍 Top Attractions in", place)
print(tool.search_attractions(place))


📍 Top Attractions in Udaipur


ValidationError: 1 validation error for TavilySearchAPIWrapper
  Value error, Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. [type=value_error, input_value={'tavily_api_key': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error